In [ ]:
# !pip install Pillow==9.1.1
# !pip install pytesseract
# !pip install bert_score
# !pip install torch
# !apt-get install tesseract-ocr

In [4]:
# from bert_score import score
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from tqdm import tqdm
import pandas as pd
import pickle
from PIL import Image
# import pytesseract
import os
import json

In [5]:
# 到資料區間路徑
os.chdir("taitra_all")

<h1 style="text-align:center;"> Data processing </h>

<h3>
    
>**data x**
>- thumbnail 文字
>- 標題 ＆ 敘述
>-  bert-score

>**data y**
>- views
>- watchtimes
>- impressions

>**feature processing**
>- clean：Nan to 0, Drop duplicates, Feature to array
>- PCA
>- T-SNE
>- binary
</h>

## Download thumbnail

In [ ]:
# from tqdm import tqdm
# import requests
# import cv2

# file = 
# df = pd.read_csv("file")
# ids = df["id"]

# for id in tqdm(ids):
#     url = f"https://img.youtube.com/vi/{id}/maxresdefault.jpg"
#     img = requests.get(url)

#     with open(f"thumbnails/{id}.jpg","wb") as file:
#         file.write(img.content)
#     file.close()

## Get thumbnail's text
- pytesseract ocr

In [ ]:
file = 
df = pd.read_csv("file")
ids = df["id"]
thumbnails_text = {}

for id in tqdm(ids.to_list()):
    image = Image.open(f'./thumbnails/{id}.jpg')
    text = pytesseract.image_to_string(image, lang='eng')
    thumbnails_text.update({f"{id}":text})
# print(thumbnails_text)
with open("thumbnails_text.json",'w', encoding="utf-8") as file:
    json.dump(thumbnails_text, file)
file.close()

## Cal thumbnail text relation with title score
- Bert score

In [ ]:
"""
計算 thumbnail text 與 title 的 score 輸出.pt file  
"""
with open('thumbnails_text.json','r',encoding='utf-8') as file:
    df = json.load(file)
file.close()

data = pd.read_csv("表格資料_510_edit.csv", index_col=False)
titles, thumb_texts = [], []
for key in tqdm(df.keys()): 
    title = data[data["影片"]==key]["影片標題"].to_string(index=False)
    thumb_text = df[key]
    titles.append(title)
    thumb_texts.append(thumb_text)

P, R, F1 = score(titles, thumb_texts, lang="others", verbose=True)
scores = {"P":P, "R":R, "F1":F1}
torch.save(scores,"bert-scores.pt")

In [ ]:
scores = torch.load("bert-scores.pt")
df = pd.read_csv('表格資料_edit_thumbnails.csv',encoding='utf-8-sig')

df['p-score'] = scores['P']
df['r-score'] = scores['R']
df['f1-score'] = scores['F1']
df.to_csv("表格資料_edit_thumbnails.csv",encoding='utf-8-sig',index=False)

## thumbnail text long

In [17]:
with open('thumbnails_text.json','r',encoding='utf-8') as file:
    df = json.load(file)
file.close()

text_len = []
for key in tqdm(df.keys()):
    chars = len(df[key])
    text_len.append(chars)

df = pd.read_csv("表格資料_edit_thumbnails.csv")
df["thumbnail_text_len"] = text_len
df.to_csv("表格資料_edit_thumbnails.csv",index=False)

344


100%|████████████████████████████████████████████████████████████████████████████████████████| 344/344 [00:00<?, ?it/s]

344


## Feature to array

In [13]:
df = pd.read_csv('表格資料_edit_thumbnails.csv',encoding='utf-8-sig')
df.drop(df[df['p-score']==0].index, axis=0, inplace=True)
# target
impressions = df.loc[:,"impressions"].values
views = df.loc[:,"views"].values
watchtimes = df.loc[:,"watchtimes(hr)"].values*3600
# 提取 features to nparray
bert_score = df.loc[:,"f1-score"].values
contrast = df.loc[:,"contrast"].values
contrast = contrast.reshape((len(contrast),1))
nlp_scores = df.loc[:,'p-score':'f1-score']

print(impressions.shape,
      #features.shape,
      contrast.shape,
      watchtimes.shape,
      views.shape,
      nlp_scores.shape
     )
# df.plot(x=nlp_scores['f1-score'])

(321,) (321, 1) (321,) (321,) (321, 3)


In [22]:
# 看最小＆最大相似度的標題
score_type = 'f1-score'
mini = df[df[score_type] == min(df[score_type])]
maximum = df[df[score_type] == max(df[score_type])]
with open('thumbnails_text.json','r',encoding='utf-8') as file:
    thumb_text = json.load(file)
file.close()

print("標題:", mini['title'].values[0], "\n縮圖:" , thumb_text[mini['id'].values[0]])
print('---------------------------------------------')
print("標題:", maximum['title'].values[0], "\n縮圖:" , thumb_text[maximum['id'].values[0]])

標題: SAN CHIA PRECISION INDUSTRY CO., LTD. 
縮圖: Bevel Gear

Featur

Easy shifting operation

The small footprint, large power is

transmitted

lubricating easy, less wear

The high and low speed can be pa

Usage:

change speed transmission of power
hange the direction of t

power

 

---------------------------------------------
標題: TK-EBLR-MULTI LAYER INFLATION MACHINE | DIING KUEN PLASTIC MACHINERY CO., LTD. | Taiwantrade 
縮圖: TK-EBLR-MULTI LAYER
INFLATION MACHINE

 

